In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:75% !important; }</style>"))

In [2]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import re
import pprint

In [3]:
def get_brands_urls(main_url="https://mobildoktoren.dk"):
    html = urlopen(main_url)
    soup = BeautifulSoup(html, "html.parser")
    # we extract the urls corresponding to each of the phones on the page of the brand
    urls = {}
    n=0
    while True:
        try:
            onclick_field = soup.find("div", id="brandborders").find_all("div")[n].get("onclick")
            brand = soup.find("div", id="brandborders").find_all("div")[n].find("img").get("alt").replace("reparationer", "").strip()
            urls[brand] = main_url + re.search("href='(.*)';$", onclick_field).group(1)
            n+=1
        except:
            break
    return urls

get_brands_urls()

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


{'Apple': 'https://mobildoktoren.dk/m/apple-reparation.html',
 'Samsung': 'https://mobildoktoren.dk/m/samsung-reparation.html',
 'Nokia': 'https://mobildoktoren.dk/m/nokia-reparation.html',
 'LG': 'https://mobildoktoren.dk/m/lg-reparation.html',
 'Sony': 'https://mobildoktoren.dk/m/sony-reparation.html',
 'Microsoft': 'https://mobildoktoren.dk/m/microsoft-reparation.html',
 'OnePlus': 'https://mobildoktoren.dk/m/oneplus-reparation.html',
 'Huawei': 'https://mobildoktoren.dk/m/huawei-reparation.html'}

In [4]:
def get_phones_urls(brand_name, brand_url, main_url="https://mobildoktoren.dk"):
    html = urlopen(brand_url)
    soup = BeautifulSoup(html, "html.parser")
    phone_names_urls = {}
    for phone_soup in soup.find_all("div", class_="tlf"):
        phone_name = brand_name + " " + phone_soup.find("a")["title"]
        phone_name = phone_name.replace("reparation", "").strip()
        phone_names_urls[phone_name] = main_url + phone_soup.find("a")["href"]
    return phone_names_urls

# get_phones_urls(urls[0])

In [5]:
def get_phone_reparations(phone_url):
    html = urlopen(phone_url)
    soup = BeautifulSoup(html, "html.parser")
    phone_repair_soup = soup.find("table", id="repprodukter").find_all("b")
    reparation_names = [] 
    prices = []
    for n, line in enumerate(phone_repair_soup):
        if n%3==0:
            reparation_names.append(line.contents[0].strip())
        if n%3==1:
            prices.append(float(line.contents[0].replace("kr.", "").strip()))
    phone_dict = {}
    for (reparation_name, price) in zip(reparation_names, prices):
        phone_dict[reparation_name] = price
    return phone_dict

# get_phone_reparations("https://mobildoktoren.dk/t/iphone-xr-reparation.html")

In [6]:
brand_urls_dict = get_brands_urls()

repair_cost_dict = {}
for brand, brand_url in brand_urls_dict.items():
    print(brand, brand_url)
    phone_urls_dict = get_phones_urls(brand, brand_url)
    for phone_item in phone_urls_dict.items():
        phone_name = phone_item[0]
        phone_url = phone_item[1]
        repair_cost_dict[phone_name] = get_phone_reparations(phone_url)
print("done")

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


Apple https://mobildoktoren.dk/m/apple-reparation.html
Samsung https://mobildoktoren.dk/m/samsung-reparation.html
Nokia https://mobildoktoren.dk/m/nokia-reparation.html
LG https://mobildoktoren.dk/m/lg-reparation.html
Sony https://mobildoktoren.dk/m/sony-reparation.html
Microsoft https://mobildoktoren.dk/m/microsoft-reparation.html
OnePlus https://mobildoktoren.dk/m/oneplus-reparation.html
Huawei https://mobildoktoren.dk/m/huawei-reparation.html
done


In [7]:
reparations_names = []
for item in repair_cost_dict.items():
    phone_name = item[0]
    reparations = item[1]
    for reparation in reparations:
        reparations_names.append(reparation)

In [9]:
def convert_reparation_name(reparation_name):
    reparation_name_transformed = reparation_name
    if re.match("Data overfÃ¸rsel", reparation_name) is not None:
        reparation_name_transformed = "data_transfer"
    if re.match(".*SWAP.*", reparation_name) is not None:
        reparation_name_transformed = "total_replacement"
    if re.match(".*Rensning.*", reparation_name) is not None:
        reparation_name_transformed = "cleaning"
    if re.match(".*SIM.*", reparation_name) is not None:
        reparation_name_transformed = "SIM"
    if re.match("(.*skÃ¦rm.*)|(.*LCD.*)|(.*display.*)|(.*skÃ¦1rm.*)|(.*SkÃ¦rm.*)|(.*skÃ¦rn.*)", reparation_name) is not None:
        reparation_name_transformed = "screen"
    if re.match(".*Softwareproblemer.*", reparation_name) is not None:
        reparation_name_transformed = "software"
    if re.match(".*GPS.*", reparation_name) is not None:
        reparation_name_transformed = "3G_4G"
    if re.match(".*Bluetooth.*", reparation_name) is not None:
        reparation_name_transformed = "bluetooth"
    if re.match(".*(W|w)ifi.*", reparation_name) is not None:
        reparation_name_transformed = "wifi"
    if re.match("(.*bagcover.*)|(.*bag Cover.*)|(.*bag ramme.*)|(.*bag cover.*)", reparation_name) is not None:
        reparation_name_transformed = "back_frame"
    if re.match("(.*USB.*)", reparation_name) is not None:
        reparation_name_transformed = "usb"
    if re.match("(.*midtramme.*)|(.*midter ramme.*)", reparation_name) is not None:
        reparation_name_transformed = "middle_frame"
    if re.match("(.*batteri.*)", reparation_name) is not None:
        reparation_name_transformed = "battery"
    if re.match("(.*midtramme.*)|(.*midter ramme.*)", reparation_name) is not None:
        reparation_name_transformed = "middle_frame"
    if re.match("(.*front kamera.*)|(.*kamera(.*)front.*)|(.*kamera(.*)forside.*)", reparation_name) is not None:
        reparation_name_transformed = "front_camera"
    if re.match("(.*mikrofon.*)", reparation_name) is not None:
        reparation_name_transformed = "microphone"
    if re.match("(.*home key.*)|(.*home knap.*)", reparation_name) is not None:
        reparation_name_transformed = "home_button"
    if re.match("(.*linse.*)", reparation_name) is not None:
        reparation_name_transformed = "camera_lens"
    if re.match(".*kamera(.*)(B|b)agside.*", reparation_name) is not None:
        reparation_name_transformed = "back_camera"
    if re.match(".*(T|t)ouch.*", reparation_name) is not None:
        reparation_name_transformed = "touch_screen"
    if re.match("(.*audio.*)|(.*jack.*)", reparation_name) is not None:
        reparation_name_transformed = "audio_port"
    if re.match("(.*hÃ¸jtaler.*)|(.*speaker)|(.*microphone.*)", reparation_name) is not None:
        reparation_name_transformed = "speaker"
    if re.match(".*glas.*", reparation_name) is not None:
        reparation_name_transformed = "glass"
    if re.match("(.*ladestik.*)|(.*opladerstik.*)", reparation_name) is not None:
        reparation_name_transformed = "charging_socket"
    if re.match("(.*oplader*)", reparation_name) is not None:
        reparation_name_transformed = "charger"
    if re.match(".*volume.*", reparation_name) is not None:
        reparation_name_transformed = "volume_button"
    if re.match("(.*power.*)|(.*on/off.*)", reparation_name) is not None:
        reparation_name_transformed = "power_button"
    if re.match("(.*ramme.*)", reparation_name) is not None:
        reparation_name_transformed = "full_frame"
    if re.match("(.*vibrator.*)", reparation_name) is not None:
        reparation_name_transformed = "vibrator"
    if re.match("(.*tÃ¦nd/sluk.*)", reparation_name) is not None:
        reparation_name_transformed = "power_cable"
    if re.match("(.*kamera dÃ¦ksel.*)", reparation_name) is not None:
        reparation_name_transformed = "camera cover"
    if re.match("(.*UndersÃ¸gelse.*)", reparation_name) is not None:
        reparation_name_transformed = "diagnose"
        
    return reparation_name_transformed

In [10]:
for phone_name, reparations in repair_cost_dict.items():
    for reparation_name, reparation_cost in reparations.copy().items():
        reparations[convert_reparation_name(reparation_name)] = reparations.pop(reparation_name)

In [11]:
 df = pd.DataFrame.from_dict(repair_cost_dict, orient='columns', dtype=None, columns=None).transpose()

In [13]:
repair_cost_dict

{'Apple Airpods': {'charging_socket': 300.0},
 'Apple iPad 10.2': {'data_transfer': 500.0,
  'software': 350.0,
  'glass': 1500.0,
  'diagnose': 325.0},
 'Apple iPad 3': {'data_transfer': 500.0,
  'charging_socket': 400.0,
  'software': 350.0,
  'battery': 800.0,
  '3G_4G': 1000.0,
  'speaker': 1000.0,
  'audio_port': 1000.0,
  'back_camera': 1000.0,
  'front_camera': 1000.0,
  'screen': 1700.0,
  'microphone': 1000.0,
  'charger': 1200.0,
  'SIM': 1000.0,
  'glass': 800.0,
  'power_cable': 1200.0,
  'wifi': 1000.0,
  'diagnose': 325.0},
 'Apple iPad 4': {'data_transfer': 500.0,
  'charging_socket': 400.0,
  'software': 350.0,
  'battery': 800.0,
  '3G_4G': 1000.0,
  'middle_frame': 1000.0,
  'speaker': 1000.0,
  'audio_port': 1000.0,
  'back_camera': 1000.0,
  'front_camera': 1000.0,
  'microphone': 1000.0,
  'charger': 1000.0,
  'SIM': 1000.0,
  'screen': 1300.0,
  'glass': 800.0,
  'power_cable': 1200.0,
  'wifi': 1000.0,
  'diagnose': 325.0},
 'Apple iPad 9.7': {'data_transfer': 50

In [12]:
pd.set_option('display.max_rows', 300)
# df[["screen"]]
df

,charging_socket,data_transfer,software,glass,diagnose,battery,3G_4G,speaker,audio_port,back_camera,...,total_replacement,bluetooth,back_frame,vibrator,volume_button,full_frame,usb,touch_screen,camera cover,power_button
Apple Airpods,300.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Apple iPad 10.2,NaN,500.0,350.0,1500.0,325.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Apple iPad 3,400.0,500.0,350.0,800.0,325.0,800.0,1000.0,1000.0,1000.0,1000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Apple iPad 4,400.0,500.0,350.0,800.0,325.0,800.0,1000.0,1000.0,1000.0,1000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Apple iPad 9.7,400.0,500.0,350.0,1500.0,325.0,1000.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Apple iPad Air,400.0,500.0,350.0,900.0,325.0,900.0,1000.0,1000.0,1000.0,1000.0,...,2870.0,1000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Apple iPad Air 2,400.0,500.0,350.0,NaN,325.0,1200.0,NaN,NaN,NaN,1000.0,...,3370.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Apple iPad Air 3 2019 10.5,NaN,500.0,350.0,NaN,325.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Apple iPad Mini,400.0,500.0,350.0,800.0,325.0,1000.0,1000.0,1000.0,1000.0,1000.0,...,NaN,1000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Apple iPad Mini 2,400.0,500.0,350.0,800.0,325.0,NaN,1000.0,1000.0,1000.0,1000.0,...,NaN,1000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
